In [191]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np
import time
import logging

import src.utils.las_utils as las_utils
import src.utils.log_utils as log_utils
import src.utils.csv_utils as csv_utils
import src.utils.ahn_utils as ahn_utils
import src.utils.bgt_utils as bgt_utils
import src.utils.clip_utils as clip_utils

import src.fusion as fusion
from src.labels import Labels
from src.pipeline import Pipeline

from src.analysis import analysis_tools

# INFO messages will be printed to console.
log_utils.reset_logger()
log_utils.add_console_logger(level=logging.DEBUG)


import matplotlib.pyplot as plt
import laspy
import open3d as o3d

from src.utils.interpolation import FastGridInterpolator
from shapely.geometry import Point, Polygon
import numpy as np
from scipy.spatial import distance

## Search Space Reduction

In [176]:
def height_filter(points, mask, ahn_tile, limit=4.5):
    """
    Returns the label mask for the given pointcloud.

    Parameters
    ----------
    points : array of shape (n_points, 3)
        The point cloud <x, y, z>.
    mask : array of shape (n_points,) with dtype=bool
        Pre-mask used to label only a subset of the points.
    ahn_tile : AHN tile dict
        The AHN tile dict for the area. 
    limit : float (default 4.5)
        The height limit conisdered by the filter

    Returns
    -------
    An array of shape (n_points,) with dtype=bool indicating which points
    should be labelled according to this fuser.
    """

    # Merge Ground and Artifact
    if 'artifact_surface' in ahn_tile.keys():
        values = ahn_tile['ground_surface'].copy()
        values[np.isnan(values)] = ahn_tile['artifact_surface'][np.isnan(values)]
    else:
        values = ahn_tile['ground_surface'].copy()

    # Interpolate AHN ground data for points
    # TODO: time consuming? (interpolates ground for all points of tile)
    fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'], values)
    ground_z = fast_z(points[mask])

    label_mask = np.zeros((len(points),), dtype=bool)
    filter_mask = (points[mask, 2] < ground_z + limit)
    label_mask[mask] = filter_mask
    
    return label_mask

In [206]:
def sampled_ground_elevation(ahn_tile):
    ground_subsample = ahn_tile['ground_surface'][::10,::10]
    mask_ids = np.isfinite(ground_subsample)
    ground_z = ground_subsample[mask_ids]
    ground_coords = np.array(np.meshgrid(ahn_tile['x'][::10], ahn_tile['y'][::10]))[:, mask_ids]
    ground_points = np.vstack((ground_coords,ground_z)).T
    return ground_points

def closest_point_cdist(point, ground_points):
    closest_index = distance.cdist(point, ground_points[:,:2]).argmin()
    return ground_points[closest_index]

def random_polygon_points(polygon, size=10, offset=0, bbox=None):
    min_x, min_y, max_x, max_y = polygon.buffer(-offset).bounds
    roof_coords = []

    # limit polygon bounds to bbox bounds
    if bbox is not None:
        ((box_x_min, box_y_max), (box_x_max, box_y_min)) = bbox
        min_x = max(min_x,box_x_min)
        min_y = max(min_y,box_y_min)
        max_x = min(max_x,box_x_max)
        max_y = min(max_y,box_y_max)

    while len(roof_coords) < size:
        x = np.random.uniform(min_x, max_x)
        y = np.random.uniform(min_y, max_y)
        if polygon.contains(Point(x,y)):
            roof_coords.append([x,y])

    return np.asarray(roof_coords)

def get_polygon_from_bbox(bbox):
    return Polygon([bbox[0],(bbox[0][0],bbox[1][1]), bbox[1],(bbox[1][0],bbox[0][1])])

# from scipy.spatial import KDTree
# def closest_point_KD(kdtree, point, ground_points):
#     dist, idx = kdtree.query(point)
#     return ground_points[idx]



In [274]:
def building_filter(points, mask, tilecode, ahn_tile, bgt_reader, offset=1.5, padding=0, min_height=4.5):
    """
    tilecode : str
        Tilecode to use for this filter.
    bgt_reader : BGTPolyReader object
        Used to load building polygons.
    ahn_reader : AHNReader object
        AHN data will be used to set a minimum height for each building polygon.
    offset : int (default: 0)
        The footprint polygon will be extended by this amount (in meters).
    padding : float (default: 0)
        Optional padding (in m) around the tile when searching for objects.
    min_height : float (default: 4.5)
        AHN min building elevation cut-off.
    """

    # 1. Create mask and get ids of non-labelled
    label_mask = np.zeros((len(points),), dtype=bool)
    if mask is None:
        mask = np.ones((len(points),), dtype=bool)
    mask_ids = np.where(mask)[0]

    # READ BGT
    building_polygons = bgt_reader.filter_tile(
                                tilecode, bgt_types=['pand'],
                                padding=padding, offset=offset,
                                merge=True)
    if len(building_polygons) == 0: 
        return label_mask

    # 2. Label buildings
    tile_bounds = las_utils.get_bbox_from_tile_code(tilecode)
    tile_polygon = get_polygon_from_bbox(tile_bounds)
    building_mask = np.zeros((len(mask_ids),), dtype=bool)
    ground_points = sampled_ground_elevation(ahn_tile)
    ahn_bld_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'], ahn_tile['building_surface']) # ..ahn_reader.interpolate()
    for polygon in building_polygons:


        if polygon.intersects(tile_polygon):

            # 2.1 Find closest AHN ground_surface point from building center
            center = np.asarray(polygon.centroid.coords)
            bld_ground_point = closest_point_cdist(center, ground_points)

            # 2.2 Calculate building height using MAX measured roof point.
            poly_points = random_polygon_points(polygon, size=20, offset=offset, bbox=tile_bounds)
            poly_height = np.nanmax(ahn_bld_z(poly_points))
            bld_height = poly_height - bld_ground_point[2]

            # 2.3 Add building to mask if above X-meters
            if bld_height > min_height:
                # TODO if there are multiple buildings we could mask the points
                # iteratively to ignore points already labelled.
                clip_mask = clip_utils.poly_clip(points[mask, :], polygon)
                building_mask = building_mask | clip_mask

    # 3. AHN building roof elevation
    label_mask[mask_ids] = building_mask

    return label_mask

In [264]:
def outlier_filter(points, mask=None, method='radius', voxelize=False, voxel_size=0.1, params=None):
    """
    The work of this region growing algorithm is based on the comparison
    of the angles between the points normals.
    The same can also be performed in Python using scipy.spatial.cKDTree
    with query_ball_tree or query.
    """

    # 1. Create mask and get ids of non-labelled
    label_mask = np.zeros(len(points),dtype=bool)
    if mask is None:
        mask = np.ones((len(points),), dtype=bool)
    mask_ids = np.where(mask)[0]

    # Convert point cloud
    coords = np.vstack((points[mask, 0], points[mask, 1], points[mask, 2])).transpose()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coords)

    # Voxelize point cloud
    if voxelize: 
        pcd, _, trace_ids = pcd.voxel_down_sample_and_trace(voxel_size=voxel_size, min_bound=pcd.get_min_bound(), max_bound=pcd.get_max_bound())
        trace_ids = np.asarray(trace_ids)

    # Remove outliers
    if method == 'voxel_radius':

        if params is None:
            params = {'nb_points':1,'radius':0.5}
        _, non_outlier_ids = pcd.remove_radius_outlier(**params)

    elif method == 'sor':

        if params is None:
            params = {'nb_neighbors':4,'std_ratio':1}
        _, non_outlier_ids = pcd.remove_statistical_outlier(**params)

    outlier_mask = np.ones(len(coords), dtype=bool)
    if voxelize:
        non_outlier_ids = trace_ids[non_outlier_ids].flatten()
    outlier_mask[non_outlier_ids] = False
    label_mask[mask_ids] = outlier_mask

    return label_mask



In [278]:
def process_tile(tilecode, in_file, out_file, ahn_reader, bgt_reader, logger=False):

    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

    # Pre-labelled Stats
    true_labels = pointcloud.label
    num_cable_poits = np.sum(true_labels==11)
    if logger and num_cable_poits > 0:
        print(f'\n{num_cable_poits} cable points in dataset. Tile {tilecode}')

    # AHN tile dict
    ahn_tile = ahn_reader.filter_tile(tilecode)

    labels = np.zeros((len(points),), dtype='uint16')
    mask = np.ones((len(labels),), dtype=bool) # _create_mask(None, labels) in pipeline.py

    # Step 1: Height Filter
    start = time.time()
    label_mask = height_filter(points, mask, ahn_tile)
    labels[label_mask] = 1
    mask[label_mask] = False
    duration = time.time() - start
    if logger:
        print(f'Processor finished in {duration:.2f}s, ' + f'{np.count_nonzero(label_mask)} points labelled.')
        if num_cable_poits > 0:
            lost = np.sum(true_labels[label_mask]==11)
            print(f'\t{lost} cable points lost.')

    # Step 2: BGT filter
    start = time.time()
    label_mask = building_filter(points, mask, tilecode, ahn_tile, bgt_reader)
    labels[label_mask] = 2
    mask[label_mask] = False
    duration = time.time() - start
    if logger:
        print(f'Processor finished in {duration:.2f}s, ' + f'{np.count_nonzero(label_mask)} points labelled.')
        if num_cable_poits > 0:
            lost = np.sum(true_labels[label_mask]==11)
            print(f'\t{lost} cable points lost.')

    # Step 3: Outlier filter
    start = time.time()
    label_mask = outlier_filter(points, mask, method='sor', voxelize=False, voxel_size=0.1)
    labels[label_mask] = 99
    mask[label_mask] = False
    duration = time.time() - start
    if logger:
        print(f'Processor finished in {duration:.2f}s, ' + f'{np.count_nonzero(label_mask)} points labelled.')
        if num_cable_poits > 0:
            lost = np.sum(true_labels[label_mask]==11)
            print(f'\t{lost} cable points lost.')

    las_utils.label_and_save_las(pointcloud, labels, out_file)

    duration = time.time() - start
    if logger:
        stats = analysis_tools.get_label_stats(labels)
        print('STATISTICS\n' + stats)
        print(f'File processed in {duration:.2f}s, ' +
                    f'output written to {out_file}.\n' + '='*20)


In [169]:
bgt_building_file = '../datasets/Valeriusplein/bgt/bgt_buildings_demo.csv'
bgt_reader = bgt_utils.BGTPolyReader(bgt_file=bgt_building_file)

ahn_data_folder = '../datasets/Valeriusplein/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

DEBUG - Caching enabled.


### Process a single file

In [ ]:
tilecode = '2387_9699'
in_file = '../datasets/Valeriusplein/pointcloud/processed_' + tilecode + '.laz'
out_file = '../datasets/Valeriusplein/pointcloud/reduced_' + tilecode + '.laz'
process_tile(tilecode,  in_file, out_file, ahn_reader, bgt_reader, logger=True)

### Process a folder of files

In [279]:
# processing batch of tiles
tile_codes = ['2386_9699','2387_9699','2387_9700']
for tilecode in tile_codes:
    in_file = '../datasets/Valeriusplein/pointcloud/processed_' + tilecode + '.laz'
    out_file = '../datasets/Valeriusplein/pointcloud/reduced_' + tilecode + '.laz'
    process_tile(tilecode, in_file, out_file, ahn_reader, bgt_reader, logger=True)


8503 cable points in dataset. Tile 2386_9699
Processor finished in 0.25s, 2154517 points labelled.
	0 cable points lost.
Processor finished in 0.14s, 467781 points labelled.
	17 cable points lost.
Remove statistical outliers: [========================================] 100%
Processor finished in 0.47s, 14650 points labelled.
	151 cable points lost.
STATISTICS
Total:                   2788762 points
Class  0, Unlabelled      151814 points ( 5.4 %)
Class  1, Ground         2154517 points (77.3 %)
Class  2, Building        467781 points (16.8 %)
Class 99, Noise            14650 points ( 0.5 %)

File processed in 0.75s, output written to ../datasets/Valeriusplein/pointcloud/reduced_2386_9699.laz.

25616 cable points in dataset. Tile 2387_9699
Processor finished in 0.39s, 4045815 points labelled.
	0 cable points lost.
Processor finished in 0.12s, 235066 points labelled.
	0 cable points lost.
Remove statistical outliers: [========================================] 100%
Processor finished in 0

**Future Idea:**

Interpolate whole tile to remove all points below groound elevation + threshold

In [ ]:
# interpolate.griddata(points, values, (grid_x, grid_y), method='linear')

### **Alternative:** AHN Building Filter (No BGT)

In [6]:
def AHN_building_filter():
    # Building Filter: ONLY AHN
    tilecode = '2407_9765'
    in_file = '../datasets/Kattenslootbrug/pointcloud/proc_ahn4_' + tilecode + '.laz'
    ahn_data_folder = '../datasets/Kattenslootbrug/ahn/'

    # Read Point Cloud
    pointcloud = las_utils.read_las(in_file)
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros((len(points),), dtype='uint16')

    # AHN Fuser
    ahn_reader = ahn_utils.NPZReader(ahn_data_folder)
    ahn_tile = ahn_reader.filter_tile(tilecode)
    fast_z = FastGridInterpolator(ahn_tile['x'], ahn_tile['y'], ahn_tile['building_surface']) # ..ahn_reader.interpolate()
    target_z = fast_z(points)

    epsilon = 0.2
    label_mask = (points[:, 2] < target_z + epsilon)
    labels[label_mask] = 0 # TARGET_LABEL 

DEBUG - Caching enabled.


In [ ]:
# X-Y Projection
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(candidate_points[:,0], candidate_points[:,1], s=0.1)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
plt.show()